In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [33]:
import cudf as pd
import cupy as cp
import cuml
from cuml.linear_model import LogisticRegression
from cuml.ensemble import RandomForestClassifier as cuRFC
from cuml.naive_bayes import MultinomialNB
from cuml.svm import SVC
from cuml.linear_model import LogisticRegression
from cuml.multiclass import MulticlassClassifier
from cuml.multiclass import OneVsRestClassifier

import matplotlib.pyplot as plt
import seaborn as sns



In [34]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import re
import string

In [35]:
import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Embedding
from keras.layers import Bidirectional
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential

In [36]:
test_data = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip',sep = '\t')
test_data.head()

In [37]:
train_data = pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', sep = '\t')
train_data.head()

In [38]:
train_data.info()

In [39]:
test_data.info()

In [40]:
train_data.describe()

In [41]:
train_data.shape

In [42]:
test_data.describe()

In [43]:
test_data.shape

Check Null Vaules

In [44]:
train_data.isnull().sum() #no missing values

In [45]:
test_data.isnull().sum() #no missing values

In [46]:
train_data['Sentiment'].unique()

In [47]:
train=train_data.to_pandas()
sns.countplot(x ='Sentiment', data = train)

In [48]:
test=test_data.to_pandas()

Text Preprocessing

In [49]:
import string
import re
string.punctuation

In [50]:
#Punctuations-remove
train['Phrase']=train['Phrase'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)).lower())
test['Phrase']=test['Phrase'].apply(lambda x: x.translate(str.maketrans('','',string.punctuation)).lower())

In [51]:
#Lowercase conversion
def convert_lower(text):
    return text.lower()

In [52]:
train['Phrase']=train['Phrase'].apply(convert_lower)
test['Phrase']=test['Phrase'].apply(convert_lower)

In [53]:
#Special characters removed
def remove_special(text):
    x=''
    
    for i in text:
        if i.isalnum():
            x=x+i
        else:
            x=x+' '
    return x

In [54]:
train['Phrase']=train['Phrase'].apply(remove_special)
test['Phrase']=test['Phrase'].apply(remove_special)

In [55]:
from nltk.corpus import stopwords

In [56]:
stopwords.words('english')

In [57]:
#Stopwords remove
def remove_stopwords(text):
    x=[]
    for i in text.split():
        
        if i not in stopwords.words('english'):
            x.append(i)
    
    y=x[:]
    x.clear()
    return y

In [58]:
train['Phrase']=train['Phrase'].apply(remove_stopwords)
test['Phrase']=test['Phrase'].apply(remove_stopwords)

In [64]:
#join the stem words together
def join_back(list_input):
    return " ".join(list_input)

In [65]:
train['Phrase']=train['Phrase'].apply(join_back)
test['Phrase']=test['Phrase'].apply(join_back)

In [66]:
#tokenize using tenserflow-keras
X_train =train['Phrase']
y_train = train['Sentiment']
tokenize = Tokenizer()
tokenize.fit_on_texts(X_train.values)

In [67]:
X_test = test['Phrase']
X_train = tokenize.texts_to_sequences(X_train)
X_test = tokenize.texts_to_sequences(X_test)

In [68]:
#add padding using tenserflow-keras
max_sequence_len = max([len(s.split()) for s in train['Phrase']])
X_train = pad_sequences(X_train, max_sequence_len,padding='pre')
X_test = pad_sequences(X_test, max_sequence_len,padding='pre')

In [69]:
print(X_train.shape)
print(X_test.shape)

Model Training CNN

In [70]:
import tensorflow as tf
embedding_dimension = 100
input_val = len(tokenize.word_index)+1
model_CNN = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_val, embedding_dimension, input_length=max_sequence_len),
    tf.keras.layers.Conv1D(128, 2, padding='same',activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 2, padding='same',activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [71]:
model_CNN.summary()

In [72]:
model_CNN.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [73]:
history_CNN=model_CNN.fit(X_train, y_train,batch_size=512, epochs=30, verbose=1)


(ACCURACY- 79.06%)

In [74]:
model_CNN.save('CNN_Model.h5')

In [75]:
accuracy_train_CNN = history_CNN.history['accuracy']
epochs = range(1,31)
plt.plot(epochs, accuracy_train_CNN, 'g', label='Training Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [76]:
predict_x_CNN=model_CNN.predict(X_test) 
classes_x_CNN=np.argmax(predict_x_CNN,axis=1)

In [77]:
cp.unique(classes_x_CNN)

In [78]:
classes_x_CNN

Logistic Regression
1)

In [79]:
log_reg = OneVsRestClassifier(LogisticRegression())

In [80]:
log_reg.fit(X_train.astype('float32'),y_train.astype('float32'))

In [81]:
y_pred_log_reg = log_reg.predict(X_test.astype('float32'))

In [82]:
y_pred_log_reg=y_pred_log_reg.astype('int32')

In [83]:
y_pred_log_reg

In [84]:
cp.unique(y_pred_log_reg)

In [85]:
#Accurcy score
cu_score_log_reg = cuml.metrics.accuracy_score(y_train, y_pred_log_reg )

In [86]:
cu_score_log_reg

(ACCURACY- 50.84%)